In [1]:
import gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from pyevn import ArmEnv
from tqdm import tqdm
from gym.wrappers import Monitor

class TrainingMonitorCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(TrainingMonitorCallback, self).__init__(verbose)
        
    
    def _on_step(self) -> bool:
        # 获取当前步数
        current_step = self.num_timesteps
        
        # 获取当前回报
        current_reward = self.locals['rewards'][0]
        
        # 获取当前动作
        current_action = self.model.predict(self.model.policy._last_observation)[0]
        
        # 使用 tqdm 显示训练进度条
        with tqdm(total=self.model.total_timesteps) as pbar:
            pbar.update(current_step)
            pbar.set_postfix({"Reward": current_reward, "Action": current_action})
        
        return True
    
# 创建自定义环境
env = ArmEnv()

# 使用 Monitor 包装器对环境进行包装
env = Monitor(env, directory="logs", force=True)

# 获取观察空间和动作空间的维度
observation_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

# 创建 PPO 模型
model = PPO("MlpPolicy", env, verbose=0)

# 使用 PPO 算法在环境中进行训练，并指定回调函数
model.learn(total_timesteps=10000, callback=TrainingMonitorCallback())

# 使用 PPO 算法在环境中进行训练
model.learn(total_timesteps=10000)

# 使用训练好的模型进行预测
obs = env.reset()
for _ in range(100):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    if done:
        obs = env.reset()

# 保存训练好的模型
model.save("ppo_model")

ValueError: could not broadcast input array from shape (921609,) into shape (921604,)